In [1]:
import molsysmt as msm

In [2]:
prmtop_file = msm.demo['pentalanine']['pentalanine.prmtop']
inpcrd_file = msm.demo['pentalanine']['pentalanine.inpcrd']

In [3]:
molsys = msm.convert([prmtop_file, inpcrd_file], to_form='molsysmt.MolSys')

file:prmtop {'item': '/home/diego/projects@dprada/MolSysMT/molsysmt/demo/pentalanine/pentalanine.prmtop', 'input_arguments': {'entity_type', 'salt_concentration', 'item', 'entity_id', 'solute_dielectric', 'hydrogen_mass', 'ewald_error_tolerance', 'solvent_dielectric', 'water_model', 'component_id', 'component_type', 'non_bonded_method', 'cutoff_distance', 'entity_name', 'implicit_solvent', 'time', 'box', 'component_name', 'partial_charge', 'flexible_constraints', 'velocities', 'switch_distance', 'kappa', 'forcefield', 'dispersion_correction', 'rigid_water', 'coordinates', 'constraints', 'formal_charge', 'structure_id'}, 'attributes_in_form': {'bond_index', 'molecule_name', 'chain_id', 'bond_id', 'bond_type', 'group_id', 'group_type', 'atom_id', 'inner_bond_index', 'bonded_atoms', 'group_index', 'atom_name', 'bond_order', 'molecule_id', 'group_name', 'box', 'chain_name', 'chain_index', 'inner_bonded_atoms', 'chain_type', 'molecule_type', 'atom_type', 'molecule_index', 'component_index'}

In [4]:
msm.info(molsys)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_peptides,n_structures
molsysmt.MolSys,5207,1722,1716,1,1716,2,1715,1,1


In [5]:
molsys.molecular_mechanics.constraints='xxx'

In [6]:
aa = msm.convert([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], to_form='molsysmt.MolecularMechanics')

molsysmt.MolSys {'item': <molsysmt.native.molsys.MolSys object at 0x7f621673bc40>, 'input_arguments': {'item'}, 'attributes_in_form': {'entity_type', 'bond_index', 'molecule_name', 'chain_id', 'entity_id', 'bond_id', 'component_id', 'bond_type', 'group_id', 'group_type', 'atom_id', 'component_type', 'bonded_atoms', 'group_index', 'atom_name', 'bond_order', 'entity_name', 'molecule_id', 'group_name', 'component_name', 'chain_name', 'chain_index', 'chain_type', 'molecule_type', 'atom_type', 'molecule_index', 'entity_index', 'coordinates', 'constraints', 'component_index'}, 'attributes_in_other_forms': {'water_model': [{'forcefield': 'AMBER14', 'water_model': 'TIP3P'}, 'molsysmt.MolecularMechanicsDict'], 'forcefield': [{'forcefield': 'AMBER14', 'water_model': 'TIP3P'}, 'molsysmt.MolecularMechanicsDict']}, 'repeated_attributes': {}, 'status_input_attributes': False, 'status_set_attributes': True}
----
molsysmt.MolecularMechanicsDict {'item': {'forcefield': 'AMBER14', 'water_model': 'TIP3P'

NotImplementedConversionError: Error in conversion from ['molsysmt.MolSys', 'molsysmt.MolecularMechanicsDict'] to molsysmt.MolecularMechanics

In [ ]:
msm.has_attribute(molsys, 'constraints')

In [ ]:
aa.constraints

In [ ]:
msm.has_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
msm.where_is_attribute([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}], 'kappa')

In [ ]:
molsys = msm.build.solvate([molsys, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                   box_shape='truncated octahedral', clearance='14.0 angstroms',
                   to_form='molsysmt.MolSys', engine="OpenMM")

In [ ]:
molsys.molecular_mechanics.flexible_constraints

In [ ]:
import numpy as np

In [ ]:
np.arange(0,10)

In [ ]:
list(range(2,0,-1))